<a href="https://colab.research.google.com/github/salarbalou/Data_Analysis_Projects/blob/main/Data_from_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (6,019 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [6]:
import cv2
import pytesseract
import numpy as np
from google.colab import files

# Load the video file
uploaded = files.upload()
cap = cv2.VideoCapture(uploaded)

# Set the ROI (Region of Interest) for the number
x, y, w, h = 100, 100, 200, 50

# Initialize the PyTesseract OCR engine
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Initialize a list to store the time and number data
data = []

# Loop over each second of the video
while cap.isOpened():
    # Get the current time in seconds
    time_sec = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
    
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Extract the ROI from the frame
    roi = frame[y:y+h, x:x+w]

    # Apply image preprocessing to improve the readability of the number
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    # Use PyTesseract to extract the number from the image
    number_str = pytesseract.image_to_string(gray, config='--psm 7')
    
    # Convert the extracted number to a numerical value
    try:
        number = float(number_str)
    except ValueError:
        number = None
        
    # Add the time and number data to the list
    data.append((time_sec, number))

# Release the video capture object
cap.release()

# Convert the data list to a NumPy array for easier manipulation
data_arr = np.array(data)

# Remove any rows where the number is None
data_arr = data_arr[data_arr[:, 1] != None]

# Save the time vs data value data to a CSV file
np.savetxt('data.csv', data_arr, delimiter=',')

error: ignored